In [1]:
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
w = np.random.randn(7, 3)
layer = MatMul(w)
h = layer.forward(c)
h

array([[ 0.34583974, -1.17097983, -0.08817798]])

In [2]:
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

w_in = np.random.randn(7, 3)
w_out = np.random.randn(3, 7)

in_layer0 = MatMul(w_in)
in_layer1 = MatMul(w_in)
out_layer = MatMul(w_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

s

array([[-0.24123514,  0.07865997, -0.51392022,  0.02251349,  0.32898575,
        -0.44202285,  0.52919999]])